# Generate Crater Shapefiles from CSV

*1. This notebook demonstrates how to create individual shapefiles for each lunar crater using GDAL and GeoPandas.*  
*2. Each crater's shapefile is created based on its center coordinates and diameter, and these information is stored in a CSV file with the following format.*  
*3. The projection used is Azimuthal Equidistant Projection for Moon 2000.*
<table align="center">
  <thead>
    <tr>
      <th style="text-align:center; vertical-align:middle;">FID</th>
      <th style="text-align:center; vertical-align:middle;">filename</th>
      <th style="text-align:center; vertical-align:middle;">diameter (km)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align:center; vertical-align:middle;">1</td>
      <td style="text-align:center; vertical-align:middle;">177_58E58_44S</td>
      <td style="text-align:center; vertical-align:middle;">25.13</td>
    </tr>
    <tr>
      <td style="text-align:center; vertical-align:middle;">2</td>
      <td style="text-align:center; vertical-align:middle;">176_86E59_82S</td>
      <td style="text-align:center; vertical-align:middle;">26.1</td>
    </tr>
    <tr>
      <td style="text-align:center; vertical-align:middle;">3</td>
      <td style="text-align:center; vertical-align:middle;">175_24E61_75S</td>
      <td style="text-align:center; vertical-align:middle;">28.61</td>
    </tr>
     <td style="text-align:center; vertical-align:middle;">...</td>
     <td style="text-align:center; vertical-align:middle;">...</td>
     <td style="text-align:center; vertical-align:middle;">...</td>
  </tbody>
</table>



## Environment Information

- Python: 3.11
- Key libraries:
  - geopandas 0.14.2
  - gdal 3.6.2
  - fiona 1.9.5
  - shapely 2.0.5
  - pyproj 3.6.1
  - rtree 1.0.1
  - pyogrio 0.10.0
  - pandas 2.2.3
  - numpy 1.26.4

## Import necessary libraries

In [1]:
import geopandas as gpd
import pandas as pd
import re
import os
from tqdm import tqdm
from shapely.geometry import Point, Polygon

## Define Function

In [2]:
# Extract coordinates from filename
def extract_coordinate(filename):
    """
    Extract longitude and latitude from the crater filename.
    
    Filenames follow the convention: <longitude><E/W><latitude><N/S>
    e.g., '120_5E30_2N' -> lon=120.5, lat=30.2
    """
    matches = list(re.finditer(r'[a-zA-Z]', filename))
    position_of_letters = [match.start() for match in matches]

    # Extract longitude
    index_of_lon = position_of_letters[0]
    letters_lon = filename[index_of_lon]
    prefix_before_lon = filename[:index_of_lon]
    lon = float(prefix_before_lon.replace('_', '.'))
    if letters_lon == 'W':
        lon = 360 - lon

    # Extract latitude
    index_of_lat = position_of_letters[1]
    letters_lat = filename[index_of_lat]
    prefix_before_lat = filename[index_of_lon + 1:index_of_lat]
    lat = float(prefix_before_lat.replace('_', '.'))
    if letters_lat == 'S':
        lat = -lat
    return lon, lat

# Function: Create a square polygon for a crater
def create_square(center, side_length):
    """
    Generate a square polygon given the center coordinates and side length (meters).
    """
    center_point = Point(center[0], center[1])
    half_side = side_length / 2.0
    coords = center_point.buffer(half_side, cap_style=3).exterior.coords
    # Order coordinates clockwise: bottom-left, bottom-right, top-right, top-left
    square_polygon = Polygon([coords[0], coords[1], coords[2], coords[3]])
    return square_polygon

# Function: Create Shapefile
def create_shapefile(output_file, center, side_length):
    """
    Generate a shapefile for a single crater using its center coordinate and side length.
    Projection: Azimuthal Equidistant for Moon_2000
    """
    polygon = create_square(center, side_length)
    projection = (
        'PROJCS["Moon_2000_Azimuthal_Equidistant",'
        'GEOGCS["GCS_Moon_2000",DATUM["D_Moon_2000",'
        'SPHEROID["Moon_2000_IAU_IAG",1737400,0]],PRIMEM["Reference_Meridian",0],'
        'UNIT["Degree",0.0174532925199433]],'
        'PROJECTION["Azimuthal_Equidistant"],'
        f'PARAMETER["latitude_of_origin",{center[1]}],'
        f'PARAMETER["central_meridian",{center[0]}],'
        'PARAMETER["false_easting",0],PARAMETER["false_northing",0],'
        'UNIT["metre",1,AUTHORITY["EPSG","9001"]],'
        'AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102037"]]'
    )
    gdf = gpd.GeoDataFrame(geometry=[polygon], crs=projection)
    gdf.to_file(output_file)

## Main Execution

In [6]:
# Load CSV containing crater info
csv_path = r'./test/craters_read_7317.csv'# replace your csv file path
data = pd.read_csv(csv_path)
file_data_name = data.iloc[:, 1]  # Crater filenames
diameter = data.iloc[:, 2]        # Crater diameters in km
idx = data.iloc[:, 0]             # Crater IDs

In [7]:
data

,FID,filename,Diameter [km]
0,1,54_74E5_56N,10.40
1,2,17_28E4_55N,10.40
2,3,29_27E18_66N,10.55
3,4,71_49E3_87S,10.50
4,5,24_12W33_75N,10.66
...,...,...,...
7312,7313,42_01E65_50N,14.88
7313,7314,49_23E67_70N,8.85
7314,7315,57_06E66_57N,17.44
7315,7316,52_52E69_62N,9.05


In [ ]:
# Generate shapefiles for each crater
for i in tqdm(range(len(file_data_name))):
    center = extract_coordinate(file_data_name[i])
    diameter_i = diameter[i] * 1000  # Convert km to meters
    num = idx[i]
    path = os.path.join(r'Crater_files',str(num),'shp')
    os.makedirs(path, exist_ok=True)
    output_file = os.path.join(path, f"{file_data_name[i]}.shp")
    create_shapefile(output_file, center, diameter_i * 1.5)  # Use 1.5x diameter as the side length

print("Shapefile generation complete!")